In [16]:
import pandas as pd
import MeCab as mc
from wordcloud import WordCloud
import os
import re

# Twitterの外部ライブラリ(GOT3)の使い方
python3 cli.py -s (検索ワード) --since "2022-12-05"(検索開始日時) --untill "2022-12-06"(検索終了日時) -o user.csv(ファイル名) --csv

In [1]:
def reduceTwitterData(rawData):
    df = pd.read_csv(rawData)
    print(df)
    df.to_csv('./TwitterData/revised'+ rawData, columns = ['date', 'time', 'timezone', 'tweet'], index = False)

In [ ]:
reduceTwitterData("cro.csv")

In [5]:
TXT_FILE = "../../rashomon.txt"

In [7]:
def strip_CRLF_from_Text(text):
    """
    改行前後が日本語文字の場合は改行を削除する．
    それ以外はスペースに置換する．
    """
    # 改行前後の文字が日本語文字の場合は改行を削除する
    plaintext = re.sub('([ぁ-んー]+|[ァ-ンー]+|[\\u4e00-\\u9FFF]+|[ぁ-んァ-ンー\\u4e00-\\u9FFF]+)(\n)([ぁ-んー]+|[ァ-ンー]+|[\\u4e00-\\u9FFF]+|[ぁ-んァ-ンー\\u4e00-\\u9FFF]+)',
                       r'\1\3',
                       text)
    # 残った改行とタブ記号はスペースに置換する
    plaintext = plaintext.replace('\n', ' ').replace('\t', ' ')
    return plaintext

In [13]:
def mecab_wakati(text):
    #t = mc.Tagger()
    t = mc.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
    node = t.parseToNode(text)
    #print(type(node))
    sent = ""
    while(node):
        # print(node.surface, node.feature)
        if node.surface != "":  # ヘッダとフッタを除外
            word_type = node.feature.split(",")[0]
            # 名詞だけをリストに追加する
            if word_type in ["名詞"]:
                 sent += node.surface + " "  # node.surface は「表層形」
            # 動詞（の原型），形容詞，副詞もリストに加えたい場合は次の２行を有効にする
            if word_type in [ "動詞", "形容詞","副詞"]:
                sent += node.feature.split(",")[6] + " " # node.feature.split(",")[6] は形態素解析結果の「原型」
        node = node.next
        if node is None:
            break
    return sent

In [15]:
f = open(TXT_FILE, encoding="utf8")
raw = f.read()
#print(raw)
text = strip_CRLF_from_Text(raw)
#print(text)
sent = mecab_wakati(text)
# フォントの保存先を指定する
font_path = "/System/Library/Fonts/ヒラギノ角ゴシック W3.ttc"  ## Mac 版はこちら
 
# WordCloud画像を生成する
wc = WordCloud(max_font_size=36, font_path=font_path).generate(sent)
wc.to_file("wc-4.png")

In [19]:
def twitterdataToWordCloud(CSV_FILE):
    df = pd.read_csv(CSV_FILE)
    tweet = df.loc[: , "tweet"]
    tweetcontent = ""
    for i in range(len(tweet)):
        tweetcontent += tweet[i]
    # f = open(tweetcontent, encoding="utf8")
    # raw = f.read()
    #print(raw)
    text = strip_CRLF_from_Text(tweetcontent)
    #print(text)
    sent = mecab_wakati(text)
    # フォントの保存先を指定する
    font_path = "/System/Library/Fonts/ヒラギノ角ゴシック W3.ttc"  ## Mac 版はこちら
    wc = WordCloud(max_font_size=36, font_path=font_path).generate(sent)
    wc.to_file("wc-4.png")

In [20]:
twitterdataToWordCloud("./TwitterData/revisedcro.csv")